# Lagrange multipliers

I need to check that I'm working out Lagrange multipliers correctly, so that I can compare convergence parameters.

Choose a test case with difference numbers of optimisation parameters and constraints, for clarity.

In [2]:
import numpy as np
import cvxpy as cp

# Set up the problem and define the expected result.
# Maximise f(x1,x2) = x1 + x2, subject to the following constraint:
# c1(x1,x2) = x1**2 + x2**2 - 1 = 0
# http://en.wikipedia.org/wiki/Lagrange_multiplier
m = 1
n = 2
# case.solver_args.x[0:2] = 1.0e0
# case.solver_args.maximise = True

# N.B. results can flip to minimum instead of maximum
# if x(1), x(2) are initialised at different points...

# Construct problem
x = cp.Variable(n)
objective = cp.Maximize(x[0] + x[1])
constraints = [x[0]**2 == 0]
prob = cp.Problem(objective, constraints)

# Solve with cvxpy to get Lagrange multipliers out
try:
    result = prob.solve()
    print(x.value)
except cp.DCPError:
    print("DCP Error")

# Expected values
# x_exp = np.array([0.5 * 2.0 ** (1 / 2), 0.5 * 2.0 ** (1 / 2)])
# objf_exp = 2.0 ** (1 / 2)
# c_exp = np.array([0.0])
# vlam_exp = np.array([1.0 * 2.0 ** (1 / 2)])



DCP Error


It appears that test case 4 isn't convex, so `cvxpy` can't handle it. Interesting...

Try test case 1:

In [3]:
import numpy as np
import cvxpy as cp

# Minimise f(x1,x2) = (x1 - 2)**2 + (x2 - 1)**2 subject to the following constraints:
# c1(x1,x2) = x1 - 2*x2 + 1 = 0
# c2(x1,x2) = -x1**2/4 - x2**2 + 1 >= 0
n = 2
m = 2

# Construct problem
x = cp.Variable(n)
objective = cp.Minimize((x[0] - 2)**2 + (x[1] - 1)**2)
constraints = [x[0] - 2*x[1] + 1 == 0, -x[0]**2/4 - x[1]**2 + 1 >= 0]
prob = cp.Problem(objective, constraints)

# Solve with cvxpy to get Lagrange multipliers out
result = prob.solve()

# Expected values
x_exp = np.array([8.228756e-1, 9.114378e-1])
objf_exp = 1.393464
vlam_exp = np.array([-1.594491, 1.846591])

# Assert we've got the expected values (same as VMCON)
np.testing.assert_allclose(x.value, x_exp)
np.testing.assert_allclose(prob.value, objf_exp, rtol=1e-6)

print("Lagrange multipliers:")
lag_mults = []
for constraint in constraints:
    lag_mults.append(constraint.dual_value)
    print(f"{constraint.dual_value}")

# Sign flipped on first value! Strange...
# np.testing.assert_allclose(lag_mults, vlam_exp)

Lagrange multipliers:
1.594491808182122
1.8465902435562314


cvxpy solves, and the solution is the same as VMCON. The Lagrange multipliers are almost the same, with the first Lagrange multiplier differing in sign.

Now use cvxpy's solution to calculate my own Lagrange multipliers:

In [4]:
# Now try alternative lag mults calc
def calc_lag_mults(cnorm, fgrd, m):
    n = fgrd.shape[0]
    constr_partials = cnorm
    # Solve n (no. of optimisation parameters) simultaneous equations to determine m (no. of constraints) Lagrange multipliers
    lag_mults = np.linalg.lstsq(constr_partials[:n, :m], fgrd, rcond=None)

    # Return solution vector only
    return lag_mults[0]

def eval_grad(x):
    """Gradient function evaluator."""

    fgrd = np.array([2.0 * (x[0] - 2.0), 2.0 * (x[1] - 1.0)])
    cnorm = np.array([[1.0, -0.5 * x[0]], [-2.0, -2.0 * x[1]]])
    return fgrd, cnorm

fgrd, cnorm = eval_grad(x.value)
my_lag_mults = calc_lag_mults(cnorm, fgrd, m)
print("My Lagrange multipliers")
print(my_lag_mults)

np.testing.assert_allclose(my_lag_mults, vlam_exp, rtol=1e-6)
# Fails due to first Lag mult sign diff
# np.testing.assert_allclose(my_lag_mults, lag_mults, rtol=1e-6)

My Lagrange multipliers
[-1.59449112  1.84659144]


## Comparison

Lagrange multipliers from VMCON, cvxpy and my own all have the same absolute value. However, the first element of the cvxpy answer has a different sign. This doesn't bother me that much: I'm interested in replicating VMCON's Lagrange multipliers using my own calculation. In this case, it works. So, why is there a difference between the test case and a full run?

## Add a bound

My suspicion is that the addition of bounds are causing the difference. The integration test cases have no bounds, whereas the regression tests do; that could explain the similarity of Lagrange multipliers in the integration case, but the difference in the regression case. To test this, I'll add a bound to a integration case.

Add a bound to integration test case 1, with cvxpy. This has to be expressed as a constraint for cvxpy:

In [5]:
# Minimise f(x1,x2) = (x1 - 2)**2 + (x2 - 1)**2 subject to the following constraints:
# c1(x1,x2) = x1 - 2*x2 + 1 = 0
# c2(x1,x2) = -x1**2/4 - x2**2 + 1 >= 0
# Add upper bound to x[0]: x[0] <= 1
# 1 - x[0] >= 0
n = 2
m = 3

# Construct problem
x = cp.Variable(n)
objective = cp.Minimize((x[0] - 2)**2 + (x[1] - 1)**2)
constraints = [x[0] - 2*x[1] + 1 == 0, -x[0]**2/4 - x[1]**2 + 1 >= 0, x[0] >= 0]
prob = cp.Problem(objective, constraints)

# Solve with cvxpy to get Lagrange multipliers out
result = prob.solve()

# Expected values
x_exp = np.array([8.228756e-1, 9.114378e-1])
objf_exp = 1.393464
vlam_exp = np.array([-1.594491, 1.846591])

# Assert we've got the expected values (same as VMCON)
np.testing.assert_allclose(x.value, x_exp)
np.testing.assert_allclose(prob.value, objf_exp, rtol=1e-6)

print("Lagrange multipliers:")
lag_mults = []
for constraint in constraints:
    lag_mults.append(constraint.dual_value)
    print(f"{constraint.dual_value}")

# Sign flipped on first value! Strange...
# np.testing.assert_allclose(lag_mults, vlam_exp)

Lagrange multipliers:
1.594491819648742
1.846591297358343
2.3919474947237467e-10


The bound was chosen so as not to affect the solution (it's not "active"). However, we now get 3 Lagrange multipliers. Using my existing calculation:

In [6]:
def eval_grad_with_bound(x):
    """Gradient function evaluator."""

    fgrd = np.array([2.0 * (x[0] - 2.0), 2.0 * (x[1] - 1.0)])
    # Add partial differentials for bound on x[0] to cnorm
    cnorm = np.array([[1.0, -0.5 * x[0], -1], [-2.0, -2.0 * x[1], 0]])
    return fgrd, cnorm

fgrd, cnorm = eval_grad_with_bound(x.value)
my_lag_mults = calc_lag_mults(cnorm, fgrd, m)
print("My Lagrange multipliers")
print(my_lag_mults)

# np.testing.assert_allclose(my_lag_mults, vlam_exp, rtol=1e-6)
# Fails due to first Lag mult sign diff
# np.testing.assert_allclose(my_lag_mults, lag_mults, rtol=1e-6)

My Lagrange multipliers
[-0.75454046  0.92502484  1.21911802]


It's clear that adding in an inactive bound causes the cvxpy and my Lagrange multipliers to go from completely agreeing to completely disagreeing.

## Adding an active bound

If a constraint is added `x[0] <= 0` that is binding (i.e. it changes the solution), what happens in this case?

In [7]:
# Minimise f(x1,x2) = (x1 - 2)**2 + (x2 - 1)**2 subject to the following constraints:
# c1(x1,x2) = x1 - 2*x2 + 1 = 0
# c2(x1,x2) = -x1**2/4 - x2**2 + 1 >= 0
# Add upper bound to x[0]: x[0] <= 0
n = 2
m = 3

# Construct problem
x = cp.Variable(n)
objective = cp.Minimize((x[0] - 2)**2 + (x[1] - 1)**2)
constraints = [x[0] - 2*x[1] + 1 == 0, -x[0]**2/4 - x[1]**2 + 1 >= 0, x[0] <= 0]
prob = cp.Problem(objective, constraints)

# Solve with cvxpy to get Lagrange multipliers out
result = prob.solve()

print("Lagrange multipliers:")
lag_mults = []
for constraint in constraints:
    lag_mults.append(constraint.dual_value)
    print(f"{constraint.dual_value}")

Lagrange multipliers:
-0.4999997899850783
7.431496512849663e-09
4.500009171412716


This bound was chosen to deliberately affect the solution (it's "active"). This has changed all 3 Lagrange multipliers, with the 2nd being very small. I suspect that the 2nd multiplier now corresponds to an "inactive" constraint, so I try removing it from the Lagrange multiplier calculation:

In [10]:
def eval_grad_with_bound(x):
    """Gradient function evaluator."""

    # Same as before
    fgrd = np.array([2.0 * (x[0] - 2.0), 2.0 * (x[1] - 1.0)])

    # Only include partial differentials constraint 1 and the bound (drop 2nd constraint)
    cnorm = np.array([[1.0, -1], [-2.0, 0]])
    return fgrd, cnorm

fgrd, cnorm = eval_grad_with_bound(x.value)
my_lag_mults = calc_lag_mults(cnorm, fgrd, m-1)
print("My Lagrange multipliers")
print(my_lag_mults)

My Lagrange multipliers
[0.5 4.5]


Interestingly, the [0] and [2] terms of the cvxpy Lagrange multipliers now equal the two multipliers produced by the independent calculation. In other words, it appears that by "dropping" the inactive constraints when calculating the Lagrange multipliers causes the cvxpy and independent calculations to align. Trying a different constraint to change what's active will validate this theory:

In [22]:
# Minimise f(x1,x2) = (x1 - 2)**2 + (x2 - 1)**2 subject to the following constraints:
# c1(x1,x2) = x1 - 2*x2 + 1 = 0
# c2(x1,x2) = -x1**2/4 - x2**2 + 1 >= 0
# Add lower bound to x[0]: x[0] >= 1.8
n = 2
m = 3

# Construct problem
x = cp.Variable(n)
objective = cp.Minimize((x[0] - 2)**2 + (x[1] - 1)**2)
constraints = [x[0] - 2*x[1] + 1 >= 0, -x[0]**2/4 - x[1]**2 + 1 >= 0, x[0] >= 1.8]
prob = cp.Problem(objective, constraints)

# Solve with cvxpy to get Lagrange multipliers out
result = prob.solve()

print("Lagrange multipliers:")
lag_mults = []
for constraint in constraints:
    lag_mults.append(constraint.dual_value)
    print(f"{constraint.dual_value}")

Lagrange multipliers:
3.0742856194040384e-10
1.2941223487459272
0.7647103085548425


With a bound of `x[0] >= 1.8`, it looks like only multipliers [1] and [2] are definitely non-zero: it looks like constraint [0] is inactive. Removing constraint [0] from the independent calculation:

In [24]:
def eval_grad_with_bound(x):
    """Gradient function evaluator."""

    # Same as before
    fgrd = np.array([2.0 * (x[0] - 2.0), 2.0 * (x[1] - 1.0)])

    # Only include partial differentials for constraint [1] and bound [0]
    # Drop constraint [0]
    cnorm = np.array([[-0.5 * x[0], 1], [-2.0 * x[1], 0]])
    return fgrd, cnorm

fgrd, cnorm = eval_grad_with_bound(x.value)
my_lag_mults = calc_lag_mults(cnorm, fgrd, m-1)
print("My Lagrange multipliers")
print(my_lag_mults)

My Lagrange multipliers
[1.29415734 0.76474161]


Again, once the inactive constraint was excluded from the independent calculation (excluded from `cnorm`), the Lagrange multipliers agree.

### Conclusion

To get the correct, identical Lagrange multipliers from VMCON, cvxpy and the independent calculation, the inactive constraints and bounds need to be excluded from each. For the independent calculation, this means excluding inactive bounds terms from the least-squares solution of the simultaneous linear equations.